In [ ]:
# Transfer Learning(전이학습)
# Transfer Learning(전이학습)이란 아주 큰 데이터셋, 즉 21,841 부류에 대해서 총 1419만7122장의 이미지로 구성되어 있는
# ImageNet 데이터를 사용해서 학습된 모델의 가중치를 가져와서, 우리가 해결하려는 문제에 맞게 보정해서 사용하는 것을 의미함
# 이때 큰 데이터셋을 사용해서 훈련된 모델을 사전학습모델(pre-trained model)이라고 함
# ImageNet 데이터의 이미지 크기는 평균적으로 469x387 이며, 이러한 2만개 이상의 부류 가운데 1000 부류만 뽑아서 데이터를 구성하고
# 정확도를 높이는 대회가 바로 ImageNet Challenge 이다

# 전이학습 필요성 
# ImageNet(풍부한 데이터) -> 사전학습모델(pre-trained model) -> 전이학습(Transfer Learning) -> 새롭게 학습되는 모델(일반적으로 파인튜닝으로 사용됨) <- 부족한 사용자 데이터

# 사전 학습 모델 구조(pre-trained model)
# 입력 -> 사전 학습된 특징 추출기(pre-trained feature extractor) -> 사전 학습된 분류기(pre-trained classifier) -> 출력

# 사전 학습된 특징 추출기(pre-trained feature extractor)
# - 특징 추출기(feature extractor)는 컨볼루션층과 풀링층의 조합으로 구성되어 있으며, ImageNet 데이터에 대해 이미 학습되어 있음
# - conv->conv->pooling->, 특징 추출기는 출력 데이터를 bottleneck 또는 feature vector 등으로 지칭함

# 사전 학습된 분류기(pre-trained classifier)
# - 분류기(classifier)는 완전 연결층으로 구성되며 추출된 특징을 입력으로 받아 최종적으로 주어진 이미지에 대한 클래스(정답)을 카테고리 형태로 분류하는 역할 수행
# - Linear->Linear->Linear(Softmax)->, 오버피팅을 줄이기 위해 출력층 이전의 Linear layer 사이에는 Dropout, BatchNormalization layer 등을 추가 할 수 있음
